In [ ]:
#******************IMPORTS AND INSTALLATIONS******************
!pip install lifetimes

import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

from lifetimes.utils import summary_data_from_transaction_data

#load the data
portfolio = pd.read_json('portfolio.json', lines=True)
profile = pd.read_json('profile.json', lines=True)
transcript = pd.read_json('transcript.json', lines=True)

#******************EXPLORE PORTFOLIO DATASET******************
print(portfolio.shape)
print(portfolio.info())
portfolio.head()


#******************EXPLORE PROFILE DATASET******************
print(profile.shape)
print(profile.info())
profile.head()

# fill in missing values
print("Number of missing values: ")
print(profile.isnull().sum())

# change the member time to day time
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# replace missing categorical values
# fill empty genders
profile['gender'] = profile['gender'].fillna('NA')
print(profile['gender'].value_counts())

print('\nDescriptive stats for age and income:')
print(profile.describe())

# visualize the income data to understand which method to use to fill in missing values
print("\nHistogram of the income feature: ")
plt.hist(profile['income'])
plt.show()

#non-longtail distribution = replace with mean

#replace missing numerical values with mean values
profile['income_na'] = profile['income'].isna().astype(int)
profile["income"] = profile["income"].fillna(profile["income"].mean())



#******************EXPLORE TRANSCRIPT DATASET******************
print(transcript.shape)
print(transcript.info())
transcript.head()

# look at different types of events
print("Unique events: ")
print(transcript['event'].unique())

# map it onto a histogram
plt.hist(transcript['event'].unique())
plt.show()

# look at different statistics for time
#print("Time data:"")
#print(transcript.describe())


#******************ONE HOT ENCODING******************
# use dummies as one hot encoding method

# use multi label binarizer
binarizer = MultiLabelBinarizer()

# use fit_transform() to fit the label sets binarizer and transform the given label sets
channel_dummies = pd.DataFrame(binarizer.fit_transform(portfolio['channels']), columns= binarizer.classes_, index=portfolio.index)

offer_df = portfolio['offer_type'].str.get_dummies()

gender_df = profile['gender'].str.get_dummies().add_prefix('gender_')

profile['year_joined'] = profile['became_member_on'].apply(lambda x: str(x.year))

year_df = profile['year_joined'].str.get_dummies().add_prefix('year_joined_')

events_df = transcript['event'].str.get_dummies()
events_df.drop('transaction', axis=1, inplace=True)

#create a new data frame for portfolio including potrfolio, channel dummies, and offer dummies
portfolio = pd.concat([portfolio, channel_dummies, offer_df], axis=1)
print(portfolio.info())
portfolio.head()

#create a new data frame for profile, including profile, gender dummies and and year joined dummies
profile = pd.concat([profile, gender_df, year_df], axis=1)

profile.drop(['became_member_on'], axis=1, inplace=True)
profile.drop(['year_joined'], axis=1, inplace=True)


#create a new data frame for transcript, including transcript and type of event dummies
transcript = pd.concat([transcript, events_df], axis=1)

# print them out
profile.head()
portfolio.head()
transcript.head()

#******************MERGING DATASETS ******************

# for offers = all the events that are not transactions
offers_data = transcript.query('event != "transaction"').copy()
offers_data['offer_id'] = offers_data['value'].apply(lambda x: list(x.values())[0])
offers_data.drop(['value'], axis=1, inplace=True)

#merge with the profile and portfolio
offers_data = offers_data.merge(profile, left_on='person', right_on='id')
offers_data = offers_data.merge(portfolio, left_on='offer_id', right_on='id')
offers_data.drop(['id_x', 'id_y'], axis=1, inplace=True)
offers_data.set_index('offer_id', inplace=True)

# remove the outliars that are over the age of 99
offers_data = offers_data.query('age <= 99')

# for transactions = events that resulted in transactions
transac_data = transcript.query('event == "transaction"').copy()
transac_data['amount'] = transac_data['value'].apply(lambda x: list(x.values())[0])
transac_data.drop(['value', 'offer completed', 'offer received', 'offer viewed'], axis=1, inplace=True)

#merge with profile
transac_data = transac_data.merge(profile, left_on='person', right_on='id')
transac_data.drop(['event'], axis=1, inplace=True)

# remove the outliars that are over the age of 99
transac_data = transac_data.query('age <= 99')

# remove the outliars that are over the age of 99
profile = profile.query('age <= 99')

offers.reset_index(inplace=True)

#visualize the data
profile.head()
print(profile.info())

transac_data.head()
print(transac_data.info())

offers_data.head()
print(offers_data.info())

# see duplicate offers in the offers data


#******************DEALING WITH DUPLICATE OFFERS******************
offers_data[offers_data.duplicated(subset=['offer_id', 'person'], keep=False)].head()

#******************ADD RECEIPT, VIEW AND COMPLETION TIMES******************
# add 0.1 so there is no 0 values for time
offers_data['time'] = offers_data['time'] + 0.1

offers_data['received_time'] = offers_data['offer_received'] * offers_data['time']
offers_data['viewed_time'] = offers_data['offer_viewed'] * offers_data['time']
offers_data['completed_time'] = offers_data['offer_completed'] * offers_data['time']

#visualize the data
offers_data.head()

#******************KEEPING TRACK OF MULTIPLE OFFERS BY THE SAME PERSON******************
min_dupes = offers_data.groupby(['person', 'offer_id', 'event']).min()
num_dupes = offers_data.groupby(['person', 'offer_id', 'event'])[['offer_received', 'offer_viewed', 'offer_completed']].sum()
min_dupes.update(num_dupes)

offers_grouped = min_dupes.groupby(['person', 'offer_id']).max().reset_index()
offers_grouped.fillna(0, inplace=True)

offers_grouped['end_time'] = offers_grouped['received_time'] + offers_grouped['duration'] * 24
offers_grouped['viewed_on_time'] = offers_grouped.apply(lambda x: 1 if x['viewed_time'] < x['end_time'] and x['viewed_time'] != 0 else 0, axis=1)
offers_grouped['completed_on_time'] = offers_grouped.apply(lambda x: 1 if x['completed_time'] < x['end_time'] and x['completed_time'] != 0 else 0, axis=1)

offers_grouped = offers_grouped.query('~(completed_time > 0 and viewed_time == 0)')
offers_grouped = offers_grouped.query('~(completed_time != 0 and completed_time < viewed_time)')
offers_grouped = offers_grouped.query('~(viewed_time != 0 and viewed_time < received_time)')

#******************CONVERTING TIME TO DATE TIME******************
transac_data['datetime'] = pd.to_datetime(transac_data['datetime'], format='%Y%m%d')

#******************FREQUENCY******************
frequency = summary_data_from_transaction_data(transac_data, 'person', 'datetime', monetary_value_col='amount')
frequency.drop('T', axis=1, inplace=True)

profile.rename({'id': 'person'}, axis=1, inplace=True)
profile.set_index('person', inplace=True)

customer_info = profile.join(frequency)

#print out the information about customers
customer_info.head()
print(customer_info.info())


#useful functions
def sum_greater_than_zero(x):
    return (x > 0).sum()

def mean_greater_than_zero(x):
    return x.replace(0, np.nan).mean()

#******************BOGOG OFFERS******************
bogo_offers = offers_grouped.query('offer type == "bogo"').groupby(['person']).agg(received_count = ('offer received', sum_greater_than_zero),
                                                                                   viewed_count = ('offer viewed', sum_greater_than_zero),
                                                                                   completed_count = ('offer completed', sum_greater_than_zero))

# calculate ratios of bogo offers that were viewed to offers received
bogo_offers['bogo_view_rate'] = bogo_offers['viewed_count'] / bogo_offers['received_count']

# calculate ratios of offers that were completed (= converted) to offers received
bogo_offers['bogo_convert_rate'] = bogo_offers['completed_count'] / bogo_offers['viewed_count']

bogo_offers.drop(['viewed_count', 'received_count', 'completed_count'], axis=1, inplace=True)

#******************DISCOUNT OFFERS******************
discount_offers = offers_grouped.query('offer type == "discount"').groupby(['person']).agg(received_count = ('offer received', sum_greater_than_zero),
                                                                                                viewed_count = ('offer viewed', sum_greater_than_zero),
                                                                                                completed_count = ('offer completed', sum_greater_than_zero),
                                                                                               )
# calculate ratios of discount offers that were viewed to offers received
discount_offers['discount_view_rate'] = discount_offers['viewed_count'] / discount_offers['received_count']
# calculate ratios of discount offers that were viewed to offers received
discount_offers['discount_convert_rate'] = discount_offers['completed_count'] / discount_offers['viewed_count']

discount_offers.drop(['viewed_count', 'received_count', 'completed_count'], axis=1, inplace=True)

#******************INFORMATIONAL OFFERS******************
info_offers = offers_grouped.query('offer type == "informational"').groupby(['person']).agg(received_count = ('offer received', sum_greater_than_zero),
                                                                                            viewed_count = ('offer viewed', sum_greater_than_zero),
                                                                                            completed_count = ('offer completed', sum_greater_than_zero),
                                                                                           )
# calculate ratios of informational offers that were viewed to offers received
info_offers['informational_view_rate'] = info_offers['viewed_count'] / info_offers['received_count']
# calculate ratios of informational offers that were completed to offers viewed
info_offers['informational_convert_rate'] = info_offers['completed_count'] / info_offers['viewed_count']

info_offers.drop(['viewed_count', 'received_count', 'completed_count'], axis=1, inplace=True)

#******************ALL OFFERS******************
all_offers = offers_grouped.query('offer type != "informational"').groupby(['person']).agg(
    received_count = ('offer received', sum_greater_than_zero),
    viewed_count = ('offer viewed', sum_greater_than_zero),
    completed_count = ('offer completed', sum_greater_than_zero),
)

# calculate ratios of all offers that were viewed to offers received
all_offers['total_view_rate'] = all_offers['viewed_count'] / all_offers['received_count']

# calculate ratios of all offers that were completed to offers viewed
all_offers['total_convert_rate'] = all_offers['completed_count'] / all_offers['viewed_count']

all_offers.drop(['offers_viewed_cnt', 'offers_received_cnt', 'offers_completed_cnt'], axis=1, inplace=True)

#join the data sets
customer_performance = customer_performance.join(bogo_offers).join(discount_offers).join(info_offers).join(all_offers)
customer_performance.fillna(0, inplace=True)

customer_performance.head()
print(customer_performance.info())


#******************CUSTOMER OFFERS******************
customer_offers = offers_grouped.groupby('person').agg(
    mean_reward_amount = ('reward', 'mean'),
    mean_difficulty = ('difficulty', 'mean'),
    email_offers_count = ('email', 'sum'),
    mobile_offers_count = ('mobile', 'sum'),
    social_cnt = ('social', 'sum'),
    web_offers_count = ('web', 'sum'),
    bogo_offers_count = ('bogo', 'sum'),
    discount_offers_count = ('discount', 'sum'),
    info_offers_count = ('informational', 'sum'),
    offers_received_count = ('offer received', sum_greater_than_zero),
    offers_viewed_count = ('offer viewed', sum_greater_than_zero),
    offers_completed_count = ('offer completed', sum_greater_than_zero),
    mean_receive_amount = ('offer received', 'mean'),
    mean_view_amount = ('offer viewed', 'mean'),
    mean_complete_amount = ('offer completed', 'mean'),
)

#join data
customer_performance = customer_performance.join(customer_offers)

customer_performance.drop(['gender', 'gender_NA', 'income_na'], axis=1, inplace=True)
customer_performance.dropna(inplace=True)

#******************DATA EXPLORATION******************
starbucks_colors = ["tan", "grey", "orange", "navy", "purple"]
starbucks_d = ["c", "m", "y"]
gender_colors = ["b", "r", "g"]
sns.set_palette(sns.color_palette(starbucks_colors))

#plot gender data
df = transac_data[['gender', 'age', 'amount', 'income']]

gen_plot = sns.countplot(x='gender', data=df)
gen_plot.set_title('Gender data: ');
sns.set_palette(sns.color_palette(starbucks_colors))

#map gender female/male
df = transac_data[['gender', 'age', 'amount', 'income']].query('gender in ["F", "M"]')
df['gender'] = df['gender'].map({'M': 'Male', 'F': 'Female'})

gen = sns.pairplot(df, hue='gender')
gen.fig.suptitle("Gender data: age and income", y=1.00);
sns.set_palette(sns.color_palette(starbucks_colors))

# plot offer by event type
df = offers.groupby(['event', 'offer_type'])['person'].count().reset_index()
df['event'] = pd.Categorical(df['event'], ["offer received", "offer viewed", "offer completed"])
df.sort_values('event', inplace=True)

axis1, axis2 = plt.subplots(ncols=3, figsize=(20, 4))

df['event'] = df['event'].map({'offer received': 'Received', 'offer viewed': 'Viewed', 'offer completed': 'Completed'})

ax = sns.barplot('event', y='person', data=df.query('offer_type == "bogo"'), palette=starbucks_d, ci=None, ax=axis2[0])
ax.set(xlabel='', ylabel='Count', title='BOGO')

ax = sns.barplot('event', y='person', data=df.query('offer_type == "discount"'), palette=starbucks_d, ci=None, ax=axis2[1])
ax.set(xlabel='', ylabel='Count', title='Discount')

ax = sns.barplot('event', y='person', data=df.query('offer_type == "informational"'), palette=starbucks_d, ci=None, ax=axis2[2])
ax.set(xlabel='', ylabel='Count', title='Informational')

axis1.suptitle('Types of Offers', size=15, y=1.05);

#******************FEATURE SCALING******************
scaler = StandardScaler().fit(customer_performance)
customers_scaled = scaler.transform(customer_performance)
customers_scaled.std(axis=0)[:5], customers_scaled.mean(axis=0)[:5]

#******************DIMENSIONALITY REDUCTION******************
pca = PCA()

X_pca = pca.fit_transform(customers_scaled)

num_components=len(pca.explained_variance_ratio_)
ind = np.arange(num_components)
vals = pca.explained_variance_ratio_
cumvals = np.cumsum(vals)

plt.figure(figsize=(10,6))

#bar plot
ax = sns.barplot(ind, vals, palette=starbucks[:1], ci=None)

#line plot
ax2 = sns.lineplot(ind, cumvals, color=starbucks[1], ci=None)
ax.grid(b=True, which='major', linewidth=0.5)

ax.set_xlabel("Principal component")
ax.set_ylabel("Variance explained (%)")
plt.title('Explained variance per principal component');
cum_expl_var_ratio = np.cumsum(pca.explained_variance_ratio_)
num_components = len(cum_expl_var_ratio[cum_expl_var_ratio <= 0.8])

pca = PCA(num_components).fit(customers_scaled)
X_pca = pca.transform(customers_scaled)
X_pca = pd.DataFrame(X_pca)
np.cumsum(pca.explained_variance_ratio_)

#******************KMEANS CLUSTERING******************
cluster_model = KMeans(n_clusters=5, random_state=28).fit(X_pca)
kmeans_clusters = cluster_model.predict(X_pca)
df = customers.copy().reset_index()
df['cluster'] = kmeans_clusters
df = df.melt(id_vars=['person', 'cluster'])
ax = sns.countplot(x='cluster', data=df)
ax.set_title('Customers by cluster');